## 분석 방향(직원수)

### 값을 기준으로 소거
- 예비유니콘 선정 된 기업 중 가장 적은 직원 수의 기업이 47명 (디스이즈엔지니어링) (출처 : 중소벤처기업부)
- 유니콘 선정 기업 중 가장 적은 직원 수는 121명


### 변화율을 기준으로 소거
- 찾아놨던 유니콘 기업들의 데이터에서 연평균 직원수 성장률을 분석 후 평균값을 기준으로 사용
- 주어진 데이터 안에서 기준에 맞는 회사 추출

In [1]:
import pandas as pd
import numpy as np
import math

unicon_data = pd.read_csv("유니콘 취합.csv", index_col=0)
data = pd.read_csv("../company_nps_data.csv",index_col=0)
pd.options.display.max_rows = 60

## 연평균 성장률을 구하기 위한 함수

In [2]:
def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods - 1)) - 1

## 유니콘 회사의 직원수 데이터를 연도별로 재정렬

In [3]:
유니콘회사_연도별_직원 = unicon_data.groupby(['회사ID','년도'])['월별_직원수'].sum()
유니콘회사_연도별_직원 = 유니콘회사_연도별_직원.unstack()
유니콘회사_연도별_직원

년도,2015,2016,2017,2018,2019
회사ID,,,,,
무신사,127,891,1112,1723,3198
쏘카,178,2095,3386,3322,3995
야놀자,267,2536,3106,4193,7442
에이프로젠,169,1197,1677,1586,1890
엘앤피코스메틱,192,1895,2438,2643,2896
옐로모바일,70,459,1806,759,211
우아한형제들,437,3472,5463,7124,10175
지피클럽,15,120,147,369,1186
쿠팡,11003,62145,64672,69548,91767


## 유니콘 기업의 데이터를 바탕으로 연평균 성장률의 최솟값 구하기

In [4]:
유니콘기업_직원_성장률 = []
for company in range(0, 11):
   유니콘기업_직원_성장률.append(cagr(유니콘회사_연도별_직원.iloc[company, 0], 유니콘회사_연도별_직원.iloc[company, -1], 5))
유니콘기업_최소_성장률 = np.min(유니콘기업_직원_성장률)
유니콘기업_최소_성장률

0.3176379776996776

## 주어진 데이터의 회사들을 연도별로 재정렬

In [5]:
연도별_직원 = data.pivot_table('연매출액', '회사ID', '년도')
연도별_직원.index = 연도별_직원.index.map(lambda x : str(x))
연도별_직원.columns = 연도별_직원.columns.map(lambda x : str(x))
연도별_직원

년도,2015,2016,2017,2018,2019
회사ID,,,,,
63,NaN,NaN,NaN,1.132878e+10,NaN
95,NaN,NaN,NaN,6.569474e+08,NaN
102,NaN,NaN,NaN,7.426589e+08,NaN
295,NaN,NaN,NaN,8.377670e+07,NaN
414,NaN,NaN,NaN,1.973277e+07,NaN
...,...,...,...,...,...
2400284,NaN,NaN,NaN,1.263406e+06,NaN
2437574,NaN,NaN,NaN,2.140160e+05,NaN
2693684,NaN,NaN,NaN,NaN,238076.0


## 데이터가 존재하는 연도 1개 이하인 기업 삭제하기

In [6]:
# 연도별_직원['기간'] = 0
# for i in range(len(연도별_직원)):
#     연도별_직원['기간'][i] = 연도별_직원.iloc[i].notnull().sum()
# 연도별_직원['기간'] = 연도별_직원['기간'] - 2
# zero_duration = 연도별_직원[연도별_직원['기간'] == 0].index
# 연도별_직원 = 연도별_직원.drop(zero_duration)
# 연도별_직원

# NaN이 아닌 값(정상적인 데이터)이 2개 미만인 행 제거
연도별_직원.dropna(thresh=2, inplace=True)
id = 연도별_직원.index
연도별_직원

년도,2015,2016,2017,2018,2019
회사ID,,,,,
14271,1199495.0,4257715.0,5005248.0,8388401.0,9039340.0
44851,NaN,939238.0,NaN,1189946.0,NaN
59533,194704.0,207762.0,124299.0,144614.0,NaN
122968,37415377.0,26858719.0,32997924.0,31514468.0,NaN
122983,24179108.0,20292461.0,9470987.0,NaN,NaN
...,...,...,...,...,...
515562,254818.0,384628.0,NaN,NaN,NaN
515837,770107.0,490090.0,NaN,NaN,NaN
516607,NaN,2586020.0,NaN,1959259.0,NaN


## 기업의 데이터에서 연평균 성장률의 평균값 구하기

In [7]:
직원_성장률 = {}
for company in id:
    start = 0
    end = 0
    count = 0
    for i in range(0, 5):
        if math.isnan(연도별_직원.loc[company][i]):
            continue
        else:
            if start == 0:
                start = i
            end = i
            count += 1
    직원_성장률[company] = cagr(연도별_직원.loc[company][start], 연도별_직원.loc[company][end], count)
직원수_성장률_분류 = dict(filter(lambda elem:elem[1]>=0.32, 직원_성장률.items()))
직원수_성장률_분류 = pd.Series(직원수_성장률_분류)

## 주어진 데이터에서 직원수가 47명 이상인 기업 선별

In [8]:
직원수_분류 = data.query("월별_직원수 > 46")
직원수_분류 = 직원수_분류.pivot_table('연매출액', '회사ID', '년도')
직원수_분류.index = 직원수_분류.index.map(lambda x : str(x))
직원수_분류.columns = 직원수_분류.columns.map(lambda x : str(x))
직원수_분류 = 직원수_분류.mean(axis=1)
직원수_분류

회사ID
63        1.132878e+10
95        6.569474e+08
102       7.426589e+08
295       8.377670e+07
414       1.973277e+07
              ...     
613233    6.468078e+07
613313    4.575581e+06
613430    1.803195e+07
630190    6.574028e+08
635082    3.606991e+07
Length: 573, dtype: float64

## 주어진 데이터에서 최소 연평균 성장률이 0.32 이상인 회사 검색

In [9]:
result = {}
for company in 직원수_성장률_분류.index:
    if company in 직원수_분류.index:
        result[company] = 직원수_성장률_분류[company]
    else:
        continue
result

{'127366': 0.4069531102838311,
 '131485': 0.41808815685782075,
 '131842': 0.7242792315637252,
 '132156': 0.3950704881914413,
 '133041': 0.5152643136941037,
 '133493': 0.5945918858461632,
 '134428': 0.3703444024918723,
 '134599': 0.35713976797564073,
 '137755': 1.0433878931064715,
 '231260': 0.330863837117376,
 '232218': 0.516409577325164,
 '232755': 0.3964437029804224,
 '296071': 0.47085572828645006,
 '297175': 0.3757995784185153,
 '380843': 0.4682095149030028,
 '382403': 0.37664637568825543,
 '394749': 0.9652061057143113,
 '403470': 0.4980828727366218,
 '404804': 0.7282388923236556,
 '420165': 0.44484360880606966,
 '441218': 0.4129867169697161,
 '441997': 0.39491522628987386,
 '470994': 1.3654763425449943,
 '471513': 0.3200783093009001}

## 최종결과
### 모든 조건에 들어맞는 기업은 24개

In [13]:
result_rate = pd.Series(result)
result_rate.sort_values(ascending = False, inplace=True)

### 직원 수의 평균이 높은 대로 출력

In [16]:
result_value = {}
for company in result.keys():
    if company in 직원수_분류.index:
        result_value[company] = 직원수_분류[company]
result_value = pd.Series(result_value)
result_value.sort_values(ascending = False, inplace=True)

In [17]:
writer = pd.ExcelWriter('result.xlsx')

result_value.to_excel(writer, sheet_name='직원수')
result_rate.to_excel(writer, sheet_name='직원수_변동률')
writer.close()